<a href="https://colab.research.google.com/github/HuanAII/HuanAII/blob/main/feedforward_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

# Devide configuration

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters

In [ ]:
input_size = 28*28
hidden_size = 500
num_classes = 10
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset

In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2021-03-25 06:07:08--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-25 06:07:09--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.1’

MNIST.tar.gz.1          [      <=>           ]  33.20M  5.01MB/s    in 18s     

2021-03-25 06:07:28 (1.80 MB/s) - ‘MNIST.tar.gz.1’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw

In [ ]:
#doan ma nay la mot vi du tieu chuan ve cach tai du lieu MNIST va chuan bi cac DataLoader trong Pytorch
#Đây là một bước quan trọng tròn việc xây dụng pipeline du lieu để huấn luyện và kiểm thử mô hình học sâu

train_dataset = torchvision.datasets.MNIST(root='./',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
# Trong do torchvision.datasetss.MNist dung để tải và xử lý tập dữ liệu MNist
# root là đường dẫn thư mục gốc để luu trữ hoặc tải dữ liệu
#Train =true thì tập dữ liệu được train sẽ tải xuống còn nếu false thì tập dữ liệu kiểm thử được tải xuống
# transforms.ToTensor() chuyển đổi định dạng thành tenso và các giá trị pixel về 0 đến 1
#download=true : Neu du lieu chua duoc tai xuong , cờ này sẽ tự động tải lập dữ liệu về internet

test_dataset = torchvision.datasets.MNIST(root='./',
                                          train=False,
                                          transform=transforms.ToTensor())


# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
#torch.utils.data.DataLoader su dụng để tạo pipeline du liệu và chia dữ liệu thành các batch có khả năng xáo trộn dữ liệu chia thanh
# moi batch và shuffle=True trộn dữ liệu để giúp mô hình tránh lệ thuộc vào thứ tự dữ liêu

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
#giong voi train_loader nhung giữ nguyên thứ tự ban đầu để việc đánh giá mô hình được thực hiện nhất quán

# Fully connected neural network with one hidden layer

In [ ]:
class NeuralNet(nn.Module):
  #ham nay dinh nghia mot lop mang nổn don gian trong pytorch sử dụng 2 tầng fully conected và hàm kích hoạt ReLu
  # tham số nn.Module nghia la claass NeuralNet kế thừa như tự động tính toán đạo hàm
  def __init__(self, input_size, hidden_size, num_classes):
    #Phuong thuc init khoi tao cua lop ,, nơi idnh nghia cac lop con va mang neron se bao gom
    #voi tham so dau vao la kich thuc input , so luong neron o tang ẩn , và số class đầu ra
    super(NeuralNet, self).__init__()


    self.fc1 = nn.Linear(input_size, hidden_size)
    #Day la tang fully connected dau tien va nho nhan kich thuoc input va chuyen doi no ra co kich thước hidden
    self.relu = nn.ReLU()
    #them tinh phi tuyen tinh cho mo hinh
    self.fc2 = nn.Linear(hidden_size, num_classes)
    #No nhan dau vao tu tang an va chuyen doi no thanh dau ra co kich thuoc num_classes

  def forward(self, x): # Day la phương thức bạn xác định luồng dữ liệu qua các tầng cua rmangj nơ ron trong quá trình dự đoán đầu ra từ đầu vào
    out = self.fc1(x) # x du qya tang dau tien
    out = self.relu(out)#luu ket qua tu tang dau tien . Dieu nay giup mo hinh học quan hẹ phi tuyen tinh
    out = self.fc2(out) #tao ra dau ra cuoi cùng

    return out#ra ve dau ra cuoi cung cua mo hinh . dieu nay se duoc su dụng để tính toán mất mát và quy trồi ngược

# Create model

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
#CrossEntropyLoss() sẽ tính toán độ lệch chuẩn
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#la mot do toi uu hoa su dung thuật toán SGD với tham số là chứa các tham số có thể huấn luyện mô hình

# Train the model

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Move tensor to the configured device
    images = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward and optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
          print ('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/600], Loss: 2.283475875854492
Epoch [1/10], Step [200/600], Loss: 2.2633490562438965
Epoch [1/10], Step [300/600], Loss: 2.2417337894439697
Epoch [1/10], Step [400/600], Loss: 2.208282232284546
Epoch [1/10], Step [500/600], Loss: 2.18100643157959
Epoch [1/10], Step [600/600], Loss: 2.1789002418518066
Epoch [2/10], Step [100/600], Loss: 2.14066743850708
Epoch [2/10], Step [200/600], Loss: 2.102980136871338
Epoch [2/10], Step [300/600], Loss: 2.087911367416382
Epoch [2/10], Step [400/600], Loss: 2.0868430137634277
Epoch [2/10], Step [500/600], Loss: 2.0339438915252686
Epoch [2/10], Step [600/600], Loss: 1.9785929918289185
Epoch [3/10], Step [100/600], Loss: 1.9783756732940674
Epoch [3/10], Step [200/600], Loss: 1.9386478662490845
Epoch [3/10], Step [300/600], Loss: 1.8822011947631836
Epoch [3/10], Step [400/600], Loss: 1.8424525260925293
Epoch [3/10], Step [500/600], Loss: 1.8966349363327026
Epoch [3/10], Step [600/600], Loss: 1.825413703918457
Epoch [4/10], Step 

# Test model

In [ ]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Accuracy: {}'.format(100 * correct / total))

Accuracy: 84.01


# Save model

In [ ]:
torch.save(model.state_dict(), 'NN_model.ckpt')